<a href="https://colab.research.google.com/github/supriyag123/PHD_Pub/blob/main/Step%201B%20-%20Consolidate%20subsequences%20and%20run%20VAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install statsmodels --upgrade
!pip install -U lingam
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import math
import plotly.graph_objects as go
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, RepeatVector, TimeDistributed, Input
from keras.models import Model
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic
import ast
from statsmodels.tsa.ar_model import AutoReg
from sklearn.metrics import mean_squared_error
from math import sqrt
from matplotlib import pyplot
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LassoCV
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import TimeSeriesSplit
from numpy import arange
import warnings
warnings.filterwarnings('ignore')
#from keras.utils import plot_model
#import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import grangercausalitytests

embed_path = '/content/drive/MyDrive/PHD/2024/iosw_files/'       # Use forward slash (/) at the end
dff=pd.DataFrame()
arr = np.array([[]])
for file in os.listdir(embed_path):
    temp = np.load(embed_path + file)

    arr = np.vstack(temp)
    df = pd.DataFrame(arr)
    dff = pd.concat([dff, df], ignore_index=True)

dff.columns =['i', 'j', 'window', 'rsme']
#drop RMSE

dff.drop('rsme', axis=1, inplace=True)

compressed_max = dff.loc[dff.groupby(["i"]).window.idxmax()]
compressed_min = dff.loc[dff.groupby(["i"]).window.idxmin()]

compressed_max.columns = ['i','j','win_max']
compressed_min.columns = ['i','j','win_min']

combined_df = compressed_min.join(compressed_max.set_index('i'), on='i', lsuffix='min', rsuffix='max')
combined_df.drop('jmin', axis=1, inplace=True)
combined_df.drop('jmax', axis=1, inplace=True)
np.save(r'/content/drive/MyDrive/PHD/2024/iosw_files/combined_iosw.npy',combined_df)

Long_train_orig = np.load(r'/content/drive/MyDrive/PHD/2024/multivariate_long_sequences-TRAIN_hourly.npy')
index = 500
#We missed i=500 from processing the iosw. So here we are dropping row with index =500
Long_train= np.delete(Long_train_orig, index, axis=0)

best_window_for_long_seq = list()
AIC = list()

for i in range(Long_train.shape[0]) :
  cur_seq = Long_train[i,:,:]
  AIC.clear()
  model = VAR(cur_seq)
  min_w = int(combined_df.iloc[i]['win_min'])
  max_w = int(combined_df.iloc[i]['win_max'])

  for m in (range(min_w,max_w+1)):
    #print(Long_train)
    try:
      results = model.fit(m)
      print('Order =', m)
      print('AIC: ', results.aic)
      print('BIC: ', results.bic)
      AIC.append(results.aic)
    except:
      AIC.append(99999)
      print('VAR could not solve row number')
      print(i, m)


  minAIC_index = AIC.index(min(AIC))+min_w
  print(AIC)
  print('Minimum lag = ', minAIC_index)
  best_window_for_long_seq.append(minAIC_index)

#OUT OF ALL LOOPS NOW
#best_window_for_long_seq now contains the best multivariate window size for each of the long sequence i
print(best_window_for_long_seq)
Window_train = np.array(best_window_for_long_seq)

np.save(r'/content/drive/MyDrive/PHD/2024/multivariate_long_sequences-TRAIN_hourly.npy',Long_train)
np.save(r'/content/drive/MyDrive/PHD/2024/multivariate_long_sequences_WINDOW-TRAIN_hourly.npy',Window_train)



Streaming output truncated to the last 5000 lines.
BIC:  -257.888436044614
Order = 63
AIC:  -333.6850651118214
BIC:  -252.1780724266325
Order = 64
AIC:  -330.73845262129583
BIC:  -247.1926953784709
Order = 65
AIC:  -327.57465928542297
BIC:  -241.95277469350702
Order = 66
AIC:  -325.3036402430606
BIC:  -237.5673439668365
Order = 67
AIC:  -322.7103490225013
BIC:  -232.82042738157614
Order = 68
AIC:  -319.8779024259012
BIC:  -227.7942084759238
Order = 69
AIC:  -313.0234504747028
BIC:  -218.70490487698575
Order = 70
AIC:  -312.43031509195
BIC:  -215.83491285772007
Order = 71
AIC:  -310.7920522283715
BIC:  -211.87687664144445
Order = 72
AIC:  -311.8822343484754
BIC:  -210.60347963308575
Order = 73
AIC:  -309.0940389056881
BIC:  -205.40704352474566
Order = 74
AIC:  -302.36550817440013
BIC:  -196.22480103035917
Order = 75
AIC:  -299.35690040665867
BIC:  -190.7162626884701
Order = 76
AIC:  -295.31850374120125
BIC:  -184.13104973656777
Order = 77
AIC:  -290.21970692675023
BIC:  -176.43798782450

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# New Section